In [2]:
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys
import warnings
warnings.filterwarnings('ignore')
# sys.path.append(os.path.abspath(".."))
from src.dataset import DatasetProcessor
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('/data/raw/Food_Delivery_Data.csv')


In [4]:
# Initialize the processor
processor = DatasetProcessor()

# Clean the data (fit and transform)
df_fe = processor.fit_transform(df)

2025-11-25 13:23:19,371 INFO - Fitting DatasetProcessor on training data...
2025-11-25 13:23:21,518 INFO - Fitted iterative imputer on numeric columns: ['Delivery_person_Age', 'Delivery_person_Ratings', 'multiple_deliveries']
2025-11-25 13:23:21,762 INFO - Fitted iterative imputer on coordinate columns: ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']
2025-11-25 13:23:21,785 INFO - Learned categorical distribution for City: 3 values
2025-11-25 13:23:21,828 INFO - Learned categorical distribution for Weatherconditions: 6 values
2025-11-25 13:23:21,849 INFO - Learned categorical distribution for Road_traffic_density: 4 values
2025-11-25 13:23:21,863 INFO - Learned categorical distribution for Festival: 2 values
2025-11-25 13:23:21,876 INFO - Learned categorical distribution for Type_of_order: 4 values
2025-11-25 13:23:21,894 INFO - Learned categorical distribution for Type_of_vehicle: 4 values


In [5]:
df_fe.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),Order_Time_Category
0,0x4607,INDORES13DEL02,37.0,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:45:00,Sunny,High,2,Snack,Motorcycle,0.0,No,Urban,24,Morning
1,0xb379,BANGRES18DEL02,34.0,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:50:00,Stormy,Jam,2,Snack,Scooter,1.0,No,Metropolitian,33,Evening
2,0x5d6d,BANGRES19DEL01,23.0,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:45:00,Sandstorms,Low,0,Drinks,Motorcycle,1.0,No,Urban,26,Morning
3,0x7a6a,COIMBRES13DEL02,38.0,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:10:00,Sunny,Medium,0,Buffet,Motorcycle,1.0,No,Metropolitian,21,Evening
4,0x70a2,CHENRES12DEL01,32.0,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:45:00,Cloudy,High,1,Snack,Scooter,1.0,No,Metropolitian,30,Afternoon


In [6]:
df_fe.describe()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Vehicle_condition,multiple_deliveries,Time_taken(min)
count,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000,45593.000000,45593,45593.000000,45593.000000,45593.0
mean,29.587194,4.631898,18.911400,76.923410,18.975064,76.987074,2022-03-13 16:32:53.987235072,1.023359,0.744386,26.294607
min,20.000000,1.000000,9.957144,72.716909,9.967144,72.778726,2022-02-11 00:00:00,0.000000,0.000000,10.0
25%,25.000000,4.500000,13.029198,73.915367,13.095801,74.006619,2022-03-04 00:00:00,0.000000,0.000000,19.0
50%,29.892613,4.700000,18.911400,76.659264,18.975064,76.740028,2022-03-15 00:00:00,1.000000,1.000000,26.0
75%,34.000000,4.800000,22.732225,78.046106,22.785049,78.107044,2022-03-27 00:00:00,2.000000,1.000000,32.0
max,50.000000,5.000000,30.914057,88.433452,31.054057,88.563452,2022-04-06 00:00:00,3.000000,3.000000,54.0
std,5.685278,0.324432,5.244871,3.360163,5.246731,3.360324,NaN,0.839065,0.566339,9.383806


In [7]:
df_fe.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Order_picked              0
Weatherconditions              0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken(min)                0
Order_Time_Category            0
dtype: int64

In [8]:
df_fe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           45593 non-null  object        
 1   Delivery_person_ID           45593 non-null  object        
 2   Delivery_person_Age          45593 non-null  float64       
 3   Delivery_person_Ratings      45593 non-null  float64       
 4   Restaurant_latitude          45593 non-null  float64       
 5   Restaurant_longitude         45593 non-null  float64       
 6   Delivery_location_latitude   45593 non-null  float64       
 7   Delivery_location_longitude  45593 non-null  float64       
 8   Order_Date                   45593 non-null  datetime64[ns]
 9   Time_Order_picked            45593 non-null  object        
 10  Weatherconditions            45593 non-null  object        
 11  Road_traffic_density         45593 non-nu

In [9]:
# Separate numerical and categorical columns
num_cols = df_fe.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df_fe.select_dtypes(exclude=[np.number]).columns.tolist()

print("\nNumerical Columns:\n", num_cols)
print("\nCategorical Columns:\n", cat_cols)




Numerical Columns:
 ['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude', 'Vehicle_condition', 'multiple_deliveries', 'Time_taken(min)']

Categorical Columns:
 ['ID', 'Delivery_person_ID', 'Order_Date', 'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density', 'Type_of_order', 'Type_of_vehicle', 'Festival', 'City', 'Order_Time_Category']
